# 1.第一个express程序

## 1.1环境要求
+ nodejs16以上版本
+ 安装了npm和typescript

## 1.2安装类库
```bash
# 初始化package.json
npm init --yes

# 安装express框架.
npm i express

# 安装express的type说明文件,方便ide提示.
npm i @types/express -save-dev

# 安装生成ts-config,配置编译参数.
tsc --init

# 安装ts-node,用于直接运行typescript
npm i ts-node
```


## 1.3添加代码
```typescript
//在根目录添加index.ts
import { Request, Response } from 'express';
const express = require('express');
const app = express();

app.get('/', (req: Request, res: Response) => {
    res.send("Hello, Godance!");
});

const port = process.env.PORT ?? 3000;
app.listen(port, () => console.log(`Server listen on port:${port}`));
```

**修改package.json,添加start命令**
```json
"scripts": {
    "start":"ts-node index.ts",
    "test": "echo \"Error: no test specified\" && exit 1"
}
```

## 1.4运行程序
```bash
npm start
```

访问浏览器`http://localhost:3000`

此时在浏览器可以看到输出结果
```text
Hello, Godance!
```

# 2.添加一些功能类库

## 2.1添加config组件
```bash
npm i config
```
在根目录中添加`config`目录,然后在`config`目录中添加`default.json`、`development.json`、`test.json`和`production.json`,每个文件内容都设置为`{}`。程序启动后会先读取`default.json`,然后根据环境变量读取`development.json`、`test.json`或`production.json`。如果不想把配置直接写在json文件中,则需要先添加`custom-environment-variables.json`,然后在对应的变量中写上环境变量名称,程序会读取环境变量中的值覆盖掉json文件中的值。



## 2.2安装`lodash`和`moment`

```bash
# 基础工具库
npm i lodash

# 处理日期的库
npm i moment
```

## 2.3安装日志组件`winston`和`morgan`
```bash
# winston可以设置多种日志保存方式
npm i winston

# morgan可以在开发中显示http请求日志
npm i morgan
```



## 2.4添加日志处理

添加`server/logging.ts`:
```typescript
const winston = require('winston');

export default function () {
    winston.add(new winston.transports.Console());
    winston.add(new winston.transports.File({ filename: 'server_log.log' }));
    winston.exceptions.handle(new winston.transports.File({ filename: 'unhandledException.log' }));
    winston.rejections.handle(new winston.transports.File({ filename: 'unhandledRejection.log' }));
}
```

然后修改`index.ts`,添加以下代码，增加日志处理
```typescript
import logging from './server/logging';
logging();

const morgan = require('morgan');

app.use(morgan(':method :url :status :res[content-length] - :response-time ms'));
```


## 2.5重构项目结构
+ 添加`controllers/index.ts`
```typescript
import { Request, Response } from "express";
const config = require('config');

export default {
    async index(req: Request, res: Response) {
    res.send(`app_name=${config.get('app_name',)}`);
        
    }
}
```

+ 添加`router/index.ts`
```typescript
import { Application } from "express";
import index from '../controllers/index';
export default function (app: Application) {
    app.use('/', index.index);
}
```

+ 添加`server/router.ts`
```typescript
import router from "../router";
import { Application } from "express";

export default function (app: Application) {
    router(app);
}
```

+ 最后修改`index.ts`文件如下:
```typescript
import logging from './server/logging';
import router from './server/router';
const morgan = require('morgan');
const express = require('express');
const app = express();

//startup
logging();
router(app);

//middlewares
app.use(morgan('tiny'));

const port = process.env.PORT ?? 3000;
app.listen(port, () => console.log(`Server listen on port:${port}`));
```


## 2.6添加`nodemon`
```bash
npm i nodemon -D
```

修改`package.json`的dev命令:`nodemon index.ts`, 这样在开发过程中